In [4]:
!pip install wordcloud

In [7]:
!pip install psycopg2

In [8]:
import psycopg2
import re
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

In [17]:
# Parâmetros de conexão ao banco de dados
db_params = {
    "host": "localhost",
    "database": "telegram2",
    "user": "postgres",
    "password": "Reve1945"
}

url_pattern = r'(https?://\S+)'

built_df = pd.DataFrame()

In [ ]:
try:
    # Conectando ao banco de dados
    conn = psycopg2.connect(**db_params)

    # Criando um cursor
    cursor = conn.cursor()

    # Exemplo de consulta SQL com paginação
    page_size = 100000  # Defina o tamanho da página desejado
    offset = 0

    total_messages_captured = 0  # Variável para contagem de mensagens capturadas

    while True:
        cursor.execute("SELECT * FROM messages_order_by_channel_utc LIMIT %s OFFSET %s", (page_size, offset))
        results = cursor.fetchall()

        if not results:
            break

        # Crie uma lista para armazenar as linhas capturadas na página atual
        new_rows = []

        for row in results:
            if re.findall(url_pattern, row[3]):
                # Adicione a linha capturada à lista de linhas da página atual
                new_rows.append([row[2], row[3], row[4]])

                # Calcule as faixas de tempo para considerar
                start_time = row[4] - timedelta(minutes = 1)
                end_time = row[4] + timedelta(minutes = 3)
    
                for i in range(results.index(row) - 1, -1, -1):
                    if row[2] == results[i][2] and start_time <= results[i][4]:
                        new_rows.append([results[i][2], results[i][3], results[i][4]])
                    else:
                        break

                for i in range(results.index(row) + 1, len(results)):
                    if row[2] == results[i][2] and results[i][4] <= end_time:
                        new_rows.append([results[i][2], results[i][3], results[i][4]])
                    else:
                        break

        # Atualize a contagem de mensagens capturadas
        total_messages_captured += len(new_rows)
        print(f"Número de mensagens já capturadas: {total_messages_captured}")

        # Crie um DataFrame a partir da lista de linhas da página atual
        df = pd.DataFrame(new_rows, columns = ["channel_id", "message_data", "message_utc"])
        # Remova as duplicatas do DataFrame, se necessário
        df = df.drop_duplicates()
        
        built_df = pd.concat([built_df, df], axis = 0)
        
        offset += page_size

    print("\n")
    print('Fim da construção do dataframe.')

    # Fechando o cursor
    cursor.close()

    # Fechando a conexão
    conn.close()

except (Exception, psycopg2.Error) as error:
    print("Erro na construção do dataframe:", error)

Número de mensagens já capturadas: 103555
Número de mensagens já capturadas: 209345


In [19]:
built_df.head()

,channel_id,message_data,message_utc
0,-1001230603327,https://www.drugs.com/drug-interactions/sars-c...,2021-10-12 16:02:34
1,-1001230603327,https://twitter.com/drobertalacerda/status/144...,2021-10-12 16:14:59
2,-1001230603327,IMUNIDADE NATURAL: \n\nCientistas identificam ...,2021-10-12 16:15:23
5,-1001230603327,https://www.ncbi.nlm.nih.gov/labs/pmc/articles...,2021-10-12 16:19:35
6,-1001230603327,,2021-10-12 16:19:36


In [ ]:
stopwords = [
    "de","a",  "o",  "que",
    "e",  "do", "da", "em",
    "um", "para",        "com","não",
    "uma","os", "no", "se",
    "na", "por","mais",        "as",
    "dos","como",        "mas","ao",
    "ele","das","à",  "seu",
    "sua","ou", "quando",      "muito",
    "nos","já", "eu", "também",
    "só", "pelo",        "pela",        "até",
    "isso",        "ela","entre",       "depois",
    "sem","mesmo",       "aos","seus",
    "quem",        "nas","me", "esse",
    "eles",        "você",        "essa",        "num",
    "nem","suas",        "meu","às",
    "minha",       "numa",        "pelos",       "elas",
    "qual",        "nós","lhe","deles",
    "essas",       "esses",       "pelas",       "este",
    "dele",        "tu", "te", "vocês",
    "vos","lhes",        "meus",        "minhas",
    "teu","tua","teus",        "tuas",
    "nosso",       "nossa",       "nossos",      "nossas",
    "dela",        "delas",       "esta",        "estes",
    "estas",       "aquele",      "aquela",      "aqueles",
    "aquelas",     "isto",        "aquilo",      "estou",
    "está",        "estamos",     "estão",       "estive",
    "esteve",      "estivemos",   "estiveram",   "estava",
    "estávamos",   "estavam",     "estivera",    "estivéramos",
    "esteja",      "estejamos",   "estejam",     "estivesse",
    "estivéssemos","estivessem",  "estiver",     "estivermos",
    "estiverem",   "hei","há", "havemos",
    "hão","houve",       "houvemos",    "houveram",
    "houvera",     "houvéramos",  "haja",        "hajamos",
    "hajam",       "houvesse",    "houvéssemos", "houvessem",
    "houver",      "houvermos",   "houverem",    "houverei",
    "houverá",     "houveremos",  "houverão",    "houveria",
    "houveríamos", "houveriam",   "sou","somos",
    "são","era","éramos",      "eram",
    "fui","foi","fomos",       "foram",
    "fora",        "fôramos",     "seja",        "sejamos",
    "sejam",       "fosse",       "fôssemos",    "fossem",
    "for","formos",      "forem",       "serei",
    "será",        "seremos",     "serão",       "seria",
    "seríamos",    "seriam",      "tenho",       "tem",
    "temos",       "tém","tinha",       "tínhamos",
    "tinham",      "tive",        "teve",        "tivemos",
    "tiveram",     "tivera",      "tivéramos",   "tenha",
    "tenhamos",    "tenham",      "tivesse",     "tivéssemos",
    "tivessem",    "tiver",       "tivermos",    "tiverem",
    "terei",       "terá",        "teremos",     "terão",
    "teria",       "teríamos",    "teriam", "pra",
    "fala", "disse", "diz", "vai"
]

In [ ]:
try:
    # Certifique-se de que o DataFrame não está vazio
    if not built_df.empty:
        # Concatenar todas as mensagens em uma única string
        all_messages = ' '.join(built_df['message_data'])

        # Criar a nuvem de palavras
        wordcloud = WordCloud(stopwords = stopword_list, 
                              normalize_plurals = False, 
                              width = 800, 
                              height = 400, 
                              background_color = 'white').generate(all_messages)

        # Exibir a nuvem de palavras usando Matplotlib
        plt.figure(figsize = (10, 5))
        plt.imshow(wordcloud, interpolation = 'bilinear')
        plt.axis('off')
        plt.show()

    else:
        print("O DataFrame está vazio. Nenhuma nuvem de palavras será gerada.")

except Exception as error:
    print("Erro ao gerar a nuvem de palavras:", error)